## Import libraries & Load Dataset

In [4]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from model_utils import save_model, load_model

In [6]:
# load dataset
df = pd.read_csv("./Dataset/processed_train.csv") 
x = df.copy()
x = x.drop(columns='stroke')
y = df['stroke']
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size= 0.2, random_state= 42)
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

6052
6052
1514
1514


## Load Model 
XGB, XGB with GridSearch, Random Forest

In [7]:
xgb = load_model('./models/xgb')
random_forest = load_model('./models/random_forest')
xgb_gs = load_model('./models/xgb_gridsearch')

## Ensemble Learning 

In [9]:
# Hard voting 
eclf1 = VotingClassifier(estimators=[('xgb', xgb),
                                    ('xgb_gridsearch', xgb_gs), 
                            ('random_forest', random_forest)], 
                                    voting='hard')
eclf1 = eclf1.fit(x_train, y_train)


C:\Users\razor\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[21:48:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 5 folds for each of 5 candidates, totalling 25 fits
C:\Users\razor\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release

In [10]:
y_pred1 = eclf1.predict(x_test)
f1_score(y_test, y_pred1)

0.988567585743107

In [12]:
# Soft voting 
eclf2 = VotingClassifier(estimators=[('xgb', xgb),
                                    ('xgb_gridsearch', xgb_gs), 
                            ('random_forest', random_forest)], 
                                    voting='soft')
eclf2 = eclf1.fit(x_test, y_test)
y_pred2 = eclf2.predict(x_test)
f1_score(y_test, y_pred2)

C:\Users\razor\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[21:54:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 5 folds for each of 5 candidates, totalling 25 fits
C:\Users\razor\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release

1.0

## Generate submission

In [11]:
# from . import generate_submission
from generate_submission import generate_submission
test_df = pd.read_csv("./Dataset/processed_test.csv")
generate_submission(eclf2, test_df, 'ensemble_soft')